# TensorFlow and TensorBoard with Regularization



## Purpose

The purpose of this lab is threefold.  

1.   to review using `TensorFlow` and `TensorBoard` for modeling and evaluation with neural networks
2.   to review using data science pipelines and cross-validation with neural networks
3.   to review using `TensorFlow` for neural network regularization

We'll be continuting our investigation of the canonical [Titanic Data Set](https://www.kaggle.com/competitions/titanic/overview) that we began [previously](https://github.com/learn-co-curriculum/enterprise-paired-nn-eval).

## The Titanic

### The Titanic and it's data



RMS Titanic was a British passenger liner built by Harland and Wolf and operated by the White Star Line. It sank in the North Atlantic Ocean in the early morning hours of 15 April 1912, after striking an iceberg during her maiden voyage from Southampton, England to New York City, USA.

Of the estimated 2,224 passengers and crew aboard, more than 1,500 died, making the sinking one of modern history's deadliest peacetime commercial marine disasters. 

Though there were about 2,224 passengers and crew members, we are given data of about 1,300 passengers. Out of these 1,300 passengers details, about 900 data is used for training purpose and remaining 400 is used for test purpose. The test data has had the survived column removed and we'll use neural networks to predict whether the passengers in the test data survived or not. Both training and test data are not perfectly clean as we'll see.

Below is a picture of the Titanic Museum in Belfast, Northern Ireland.

In [12]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://upload.wikimedia.org/wikipedia/commons/c/c0/Titanic_Belfast_HDR.jpg", width=400, height=400)

### Data Dictionary

*   *Survival* : 0 = No, 1 = Yes
*   *Pclass* : A proxy for socio-economic status (SES)
  *   1st = Upper
  *   2nd = Middle
  *   3rd = Lower
*   *sibsp* : The number of siblings / spouses aboard the Titanic
  *   Sibling = brother, sister, stepbrother, stepsister
  *   Spouse = husband, wife (mistresses and fiancés were ignored)
*   *parch* : The # of parents / children aboard the Titanic
  *   Parent = mother, father
  *   Child = daughter, son, stepdaughter, stepson
  *   Some children travelled only with a nanny, therefore *parch*=0 for them.
*   *Ticket* : Ticket number
*   *Fare* : Passenger fare (British pounds)
*   *Cabin* : Cabin number embarked
*   *Embarked* : Port of Embarkation
  *   C = Cherbourg (now Cherbourg-en-Cotentin), France
  *   Q = Queenstown (now Cobh), Ireland
  *   S = Southampton, England
*   *Name*, *Sex*, *Age* (years) are all self-explanatory

## Libraries and the Data



### Importing libraries

In [13]:
# Load the germane libraries

import pandas as pd
import numpy as np
import seaborn as sns 
from pandas._libs.tslibs import timestamps
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler

import tensorflow as tf
import keras 
from keras import models
from sklearn.impute import SimpleImputer
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.losses import binary_crossentropy
from sklearn.model_selection import GridSearchCV
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from keras.wrappers.scikit_learn import KerasClassifier

# Load the TensorBoard notebook extension and related libraries
%load_ext tensorboard
import datetime

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


### Loading the data

In [14]:
# Load the data

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# We need to do this for when we mamke our predictions from the test data at the end
ids = test[['PassengerId']]

## EDA and Preprocessing

### Exploratory Data Analysis

You have already performed EDA on this data set. Look back on what you did before or see [here](https://github.com/learn-co-curriculum/enterprise-paired-nn-eval).

Of course, feel free to re-run what you have done before or try out some other EDA as you find useful.

### Preprocessing

Let's do the same prepricessing as before.

In [15]:
# Performing preprocessing on the train and test data will be more effecient if we combine the two date sets.
combined = pd.concat([train, test], axis=0, sort=False)

#Age column
combined['Age'].fillna(combined['Age'].median(),inplace=True) # Age

# Embarked column
combined['Embarked'].fillna(combined['Embarked'].value_counts().index[0], inplace=True) # Embarked
combined['Fare'].fillna(combined['Fare'].median(),inplace=True)

# Class column
d = {1:'1st',2:'2nd',3:'3rd'} #Pclass
combined['Pclass'] = combined['Pclass'].map(d) #Pclass

# Making Age into adult (1) and child (0)
combined['Child'] = combined['Age'].apply(lambda age: 1 if age>=18 else 0) 

# Break up the string that has the title and names
combined['Title'] = combined['Name'].str.split('.').str.get(0)  # output : 'Futrelle, Mrs'
combined['Title'] = combined['Title'].str.split(',').str.get(1) # output : 'Mrs '
combined['Title'] = combined['Title'].str.strip()               # output : 'Mrs'
combined.groupby('Title').count()

# Replace the French titles with Enlgish
french_titles = ['Don', 'Dona', 'Mme', 'Ms', 'Mra','Mlle']
english_titles = ['Mr', 'Mrs','Mrs','Mrs','Mrs','Miss']
for i in range(len(french_titles)):
    for j in range(len(english_titles)):
        if i == j:
            combined['Title'] = combined['Title'].str.replace(french_titles[i],english_titles[j])

# Seperate the titles into "major" and "others", the latter would be, e.g., Reverend
major_titles = ['Mr','Mrs','Miss','Master']
combined['Title'] = combined['Title'].apply(lambda title: title if title in major_titles else 'Others')

#Dropping the Irrelevant Columns
combined.drop(['PassengerId','Name','Ticket','Cabin'], axis=1, inplace=True)

# Getting Dummy Variables and Dropping the Original Categorical Variables
categorical_vars = combined[['Pclass','Sex','Embarked','Title','Child']] # Get Dummies of Categorical Variables
dummies = pd.get_dummies(categorical_vars,drop_first=True)
combined = combined.drop(['Pclass','Sex','Embarked','Title','Child'],axis=1)
combined = pd.concat([combined, dummies],axis=1)

# Separating the data back into train and test sets
test = combined[combined['Survived'].isnull()].drop(['Survived'],axis=1)
train = combined[combined['Survived'].notnull()]

# Training
X_train = train.drop(['Survived'],axis=1)
y_train = train['Survived']

# Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
test = sc.fit_transform(test)

## Neural Network Model

### Building the model

#### Define the model as a pipeline

Let's use the data science pipeline for our neural network model.

As you are now using regularization to guard against high variance, i.e. overfitting the data, in the definition of the model below include *dropout* and/or *l2* regularization. Also, feel free to experiment with different activation functions.

In [16]:
# It will help to define our model in terms of a pipeline
def build_classifier(optimizer):
# insert Sequential and layers here
    classifier = Sequential()
    classifier.add(Dense(12, activation='relu', input_dim=14))
    classifier.add(Dense(5, activation='relu'))
    classifier.add(Dense(1, activation='relu'))
    classifier.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

#### Use grid search to find help you tune the parameters

You can play with optimizers, epochs, and batch sizes. The ones that we're suggesting are not necessarily the best.

In [6]:
# Grid Search
classifier = KerasClassifier(build_fn = build_classifier)
param_grid = dict(optimizer = ['Adam'],
                  epochs=[10, 20, 50],
                  batch_size=[16, 25, 32])
grid = GridSearchCV(estimator=classifier, param_grid=param_grid, scoring='accuracy')
grid_result = grid.fit(X_train, y_train)
best_parameters = grid.best_params_
best_accuracy = grid.best_score_

Epoch 1/10
45/45 [==============================] - 0s 2ms/step - loss: 2.1709 - accuracy: 0.6278
Epoch 2/10
45/45 [==============================] - 0s 2ms/step - loss: 1.6420 - accuracy: 0.6798
Epoch 3/10
45/45 [==============================] - 0s 2ms/step - loss: 1.4536 - accuracy: 0.6938: 0s - loss: 1.3814 - accuracy: 0.70
Epoch 4/10
45/45 [==============================] - 0s 2ms/step - loss: 1.3503 - accuracy: 0.6966
Epoch 5/10
45/45 [==============================] - 0s 2ms/step - loss: 1.2654 - accuracy: 0.7079
Epoch 6/10
45/45 [==============================] - 0s 2ms/step - loss: 1.2336 - accuracy: 0.7247
Epoch 7/10
45/45 [==============================] - 0s 2ms/step - loss: 1.0804 - accuracy: 0.7683
Epoch 8/10
45/45 [==============================] - 0s 2ms/step - loss: 1.0422 - accuracy: 0.7865
Epoch 9/10
45/45 [==============================] - 0s 2ms/step - loss: 0.9593 - accuracy: 0.7851
Epoch 10/10
45/45 [==============================] - 0s 2ms/step - loss: 0.9325 - 

45/45 [==============================] - ETA: 0s - loss: 1.0241 - accuracy: 0.81 - 0s 2ms/step - loss: 1.0907 - accuracy: 0.7994
Epoch 8/20
45/45 [==============================] - 0s 2ms/step - loss: 1.0295 - accuracy: 0.7966
Epoch 9/20
45/45 [==============================] - 0s 2ms/step - loss: 0.9668 - accuracy: 0.8036
Epoch 10/20
45/45 [==============================] - 0s 2ms/step - loss: 0.9545 - accuracy: 0.8093
Epoch 11/20
45/45 [==============================] - 0s 2ms/step - loss: 0.9484 - accuracy: 0.8121
Epoch 12/20
45/45 [==============================] - 0s 2ms/step - loss: 0.9437 - accuracy: 0.8135
Epoch 13/20
45/45 [==============================] - 0s 2ms/step - loss: 0.9072 - accuracy: 0.8149
Epoch 14/20
45/45 [==============================] - 0s 2ms/step - loss: 0.8824 - accuracy: 0.8135
Epoch 15/20
45/45 [==============================] - 0s 2ms/step - loss: 0.8778 - accuracy: 0.8191
Epoch 16/20
45/45 [==============================] - 0s 2ms/step - loss: 0.8748 -

45/45 [==============================] - 0s 2ms/step - loss: 1.2637 - accuracy: 0.7303
Epoch 10/50
45/45 [==============================] - 0s 2ms/step - loss: 1.0540 - accuracy: 0.7640
Epoch 11/50
45/45 [==============================] - 0s 2ms/step - loss: 0.9940 - accuracy: 0.7739
Epoch 12/50
45/45 [==============================] - 0s 2ms/step - loss: 0.9616 - accuracy: 0.7725
Epoch 13/50
45/45 [==============================] - 0s 2ms/step - loss: 0.9524 - accuracy: 0.7725
Epoch 14/50
45/45 [==============================] - 0s 2ms/step - loss: 0.9465 - accuracy: 0.7739
Epoch 15/50
45/45 [==============================] - 0s 2ms/step - loss: 0.9407 - accuracy: 0.7767
Epoch 16/50
45/45 [==============================] - 0s 2ms/step - loss: 0.8721 - accuracy: 0.7809
Epoch 17/50
45/45 [==============================] - 0s 2ms/step - loss: 0.8557 - accuracy: 0.7837
Epoch 18/50
45/45 [==============================] - 0s 2ms/step - loss: 0.8506 - accuracy: 0.7879
Epoch 19/50
45/45 [===

Epoch 42/50
45/45 [==============================] - 0s 2ms/step - loss: 0.4488 - accuracy: 0.8359
Epoch 43/50
45/45 [==============================] - 0s 2ms/step - loss: 0.4480 - accuracy: 0.8401
Epoch 44/50
45/45 [==============================] - 0s 2ms/step - loss: 0.4489 - accuracy: 0.8401
Epoch 45/50
45/45 [==============================] - 0s 2ms/step - loss: 0.4587 - accuracy: 0.8415
Epoch 46/50
45/45 [==============================] - 0s 2ms/step - loss: 0.4492 - accuracy: 0.8401
Epoch 47/50
45/45 [==============================] - 0s 2ms/step - loss: 0.4462 - accuracy: 0.8415
Epoch 48/50
45/45 [==============================] - 0s 2ms/step - loss: 0.4430 - accuracy: 0.8429
Epoch 49/50
45/45 [==============================] - 0s 2ms/step - loss: 0.4410 - accuracy: 0.8401
Epoch 50/50
45/45 [==============================] - ETA: 0s - loss: 0.4798 - accuracy: 0.83 - 0s 2ms/step - loss: 0.4383 - accuracy: 0.8415
Epoch 1/50
45/45 [==============================] - 0s 2ms/step - l

45/45 [==============================] - 0s 2ms/step - loss: 0.7903 - accuracy: 0.7475
Epoch 25/50
45/45 [==============================] - 0s 2ms/step - loss: 0.7862 - accuracy: 0.7489
Epoch 26/50
45/45 [==============================] - 0s 2ms/step - loss: 0.7677 - accuracy: 0.7532
Epoch 27/50
45/45 [==============================] - 0s 2ms/step - loss: 0.7499 - accuracy: 0.7574
Epoch 28/50
45/45 [==============================] - 0s 2ms/step - loss: 0.7345 - accuracy: 0.7574
Epoch 29/50
45/45 [==============================] - 0s 2ms/step - loss: 0.7229 - accuracy: 0.7616
Epoch 30/50
45/45 [==============================] - 0s 2ms/step - loss: 0.7147 - accuracy: 0.7602
Epoch 31/50
45/45 [==============================] - 0s 2ms/step - loss: 0.7101 - accuracy: 0.7602
Epoch 32/50
45/45 [==============================] - 0s 2ms/step - loss: 0.7068 - accuracy: 0.7588
Epoch 33/50
45/45 [==============================] - 0s 2ms/step - loss: 0.7030 - accuracy: 0.7644
Epoch 34/50
45/45 [===

...

23/23 [==============================] - 0s 941us/step - loss: 2.3390 - accuracy: 0.6311
Epoch 6/10
23/23 [==============================] - 0s 1ms/step - loss: 2.0728 - accuracy: 0.6297
Epoch 7/10
23/23 [==============================] - 0s 968us/step - loss: 1.9086 - accuracy: 0.6297
Epoch 8/10
23/23 [==============================] - 0s 939us/step - loss: 1.8018 - accuracy: 0.6452
Epoch 9/10
23/23 [==============================] - ETA: 0s - loss: 2.3474 - accuracy: 0.56 - 0s 949us/step - loss: 1.7454 - accuracy: 0.6494
Epoch 10/10
23/23 [==============================] - 0s 813us/step - loss: 1.6195 - accuracy: 0.6592
Epoch 1/10
23/23 [==============================] - 0s 1ms/step - loss: 4.8327 - accuracy: 0.6045
Epoch 2/10
23/23 [==============================] - 0s 1ms/step - loss: 4.3590 - accuracy: 0.6129
Epoch 3/10
23/23 [==============================] - 0s 1ms/step - loss: 3.8924 - accuracy: 0.6171
Epoch 4/10
23/23 [==============================] - 0s 1ms/step - loss: 3.60

23/23 [==============================] - 0s 1ms/step - loss: 1.9625 - accuracy: 0.7391
Epoch 8/20
23/23 [==============================] - 0s 1ms/step - loss: 1.9513 - accuracy: 0.7391
Epoch 9/20
23/23 [==============================] - 0s 1ms/step - loss: 1.9443 - accuracy: 0.7461
Epoch 10/20
23/23 [==============================] - 0s 1ms/step - loss: 1.9412 - accuracy: 0.7475
Epoch 11/20
23/23 [==============================] - 0s 1ms/step - loss: 1.9376 - accuracy: 0.7447
Epoch 12/20
23/23 [==============================] - 0s 1ms/step - loss: 1.9226 - accuracy: 0.7489
Epoch 13/20
23/23 [==============================] - 0s 2ms/step - loss: 1.8552 - accuracy: 0.7602
Epoch 14/20
23/23 [==============================] - 0s 1ms/step - loss: 1.8512 - accuracy: 0.7644
Epoch 15/20
23/23 [==============================] - 0s 1ms/step - loss: 1.8420 - accuracy: 0.7672
Epoch 16/20
23/23 [==============================] - 0s 1ms/step - loss: 1.8240 - accuracy: 0.7672
Epoch 17/20
23/23 [=====

23/23 [==============================] - 0s 2ms/step - loss: 0.7205 - accuracy: 0.8230
Epoch 30/50
23/23 [==============================] - 0s 2ms/step - loss: 0.6414 - accuracy: 0.8287
Epoch 31/50
23/23 [==============================] - 0s 2ms/step - loss: 0.5816 - accuracy: 0.8315
Epoch 32/50
23/23 [==============================] - 0s 1ms/step - loss: 0.5395 - accuracy: 0.8287
Epoch 33/50
23/23 [==============================] - 0s 1ms/step - loss: 0.5151 - accuracy: 0.8258
Epoch 34/50
23/23 [==============================] - 0s 1ms/step - loss: 0.5081 - accuracy: 0.8287
Epoch 35/50
23/23 [==============================] - 0s 2ms/step - loss: 0.5049 - accuracy: 0.8202
Epoch 36/50
23/23 [==============================] - 0s 2ms/step - loss: 0.5026 - accuracy: 0.8230
Epoch 37/50
23/23 [==============================] - 0s 1ms/step - loss: 0.4995 - accuracy: 0.8258
Epoch 38/50
23/23 [==============================] - 0s 2ms/step - loss: 0.4980 - accuracy: 0.8287
Epoch 39/50
23/23 [===

23/23 [==============================] - 0s 976us/step - loss: 1.1757 - accuracy: 0.7574
Epoch 12/50
23/23 [==============================] - 0s 1ms/step - loss: 1.1699 - accuracy: 0.7588
Epoch 13/50
23/23 [==============================] - 0s 935us/step - loss: 1.1645 - accuracy: 0.7588
Epoch 14/50
23/23 [==============================] - 0s 914us/step - loss: 1.1590 - accuracy: 0.7602
Epoch 15/50
23/23 [==============================] - 0s 1ms/step - loss: 1.1545 - accuracy: 0.7616
Epoch 16/50
23/23 [==============================] - 0s 928us/step - loss: 1.1507 - accuracy: 0.7672
Epoch 17/50
23/23 [==============================] - 0s 894us/step - loss: 1.1466 - accuracy: 0.7686
Epoch 18/50
23/23 [==============================] - 0s 895us/step - loss: 1.1441 - accuracy: 0.7742
Epoch 19/50
23/23 [==============================] - 0s 882us/step - loss: 1.1410 - accuracy: 0.7728
Epoch 20/50
23/23 [==============================] - 0s 907us/step - loss: 1.1096 - accuracy: 0.7728
Epoch 

23/23 [==============================] - 0s 1ms/step - loss: 1.2468 - accuracy: 0.8191
Epoch 43/50
23/23 [==============================] - 0s 1ms/step - loss: 1.2461 - accuracy: 0.8205
Epoch 44/50
23/23 [==============================] - 0s 1ms/step - loss: 1.2451 - accuracy: 0.8191
Epoch 45/50
23/23 [==============================] - 0s 1ms/step - loss: 1.2441 - accuracy: 0.8205
Epoch 46/50
23/23 [==============================] - 0s 1ms/step - loss: 1.2437 - accuracy: 0.8205
Epoch 47/50
23/23 [==============================] - 0s 1ms/step - loss: 1.2432 - accuracy: 0.8191
Epoch 48/50
23/23 [==============================] - 0s 871us/step - loss: 1.2424 - accuracy: 0.8205
Epoch 49/50
23/23 [==============================] - 0s 921us/step - loss: 1.2419 - accuracy: 0.8219
Epoch 50/50
23/23 [==============================] - 0s 958us/step - loss: 1.2416 - accuracy: 0.8233
Epoch 1/50
23/23 [==============================] - 0s 1ms/step - loss: 5.3871 - accuracy: 0.6003
Epoch 2/50
23/23 

#### `TensorBoard`

`TensorBoard` is `TensorFlow`'s visualization toolkit. It is a dashboard that provides visualization and tooling that is needed for machine learning experimentation. The code immediately below will allow us to use TensorBoard.

N.B. When we loaded the libraries, we loaded the TensorBoard notebook extension. (It is the last line of code in the first code chunk.)

In [8]:
# Clear out any prior log data.
!rm -rf logs
# Be careful not to run this command if already have trained your model and you want to use TensorBoard.

# Sets up a timestamped log directory
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(log_dir)


# The callback function, which will be called in the fit()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#### Fitting the optimal model and evaluating with `TensorBoaard`

Define the early stopping callback. Use your best values from grid serarch with `KerasClassifer` and finally fit the model.

In [9]:
# Define the EarlyStopping object
early_stop = EarlyStopping(monitor='val_loss', min_delta=1e-8,
                           verbose=1, patience=5,
                           mode='min')

# Using KerasClassifier
classifier = KerasClassifier(build_fn = build_classifier,
                             optimizer=best_parameters['optimizer'],
                             batch_size=best_parameters['batch_size'],
                             epochs=best_parameters['epochs'])

# Fit the model with the tensorboard_callback
classifier.fit(X_train,
               y_train,
               verbose=1,
               callbacks=[early_stop, tensorboard_callback])


# Warning: If verbose = 0 (silent) or 2 (one line per epoch), then on TensorBoard's Graphs tab there will be an error.
# The other tabs in TensorBoard will still be function, but if you want the graphs then verbose needs to be 1 (progress bar).

Epoch 1/20
 1/56 [..............................] - ETA: 0s - loss: 3.3865 - accuracy: 0.6875WARNING:tensorflow:From C:\Users\bmartinezmedina\AppData\Local\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
56/56 [==============================] - 0s 3ms/step - loss: 4.1868 - accuracy: 0.6263
Epoch 2/20
56/56 [==============================] - 0s 838us/step - loss: 3.3217 - accuracy: 0.6521
Epoch 3/20
56/56 [==============================] - 0s 929us/step - loss: 2.3659 - accuracy: 0.6667
Epoch 4/20
56/56 [==============================] - 0s 810us/step - loss: 1.8585 - accuracy: 0.6857
Epoch 5/20
56/56 [==============================] - 0s 731us/step - loss: 1.7575 - accuracy: 0.7059
Epoch 6/20
56/56 [==============================] - 0s 659us/step - loss: 1.5637 - accuracy: 0.7138

In [17]:
# Call TensorBoard within SaturnCloud [Comment this out if you are not in SaturnCloud]
import os
print(f"https://{os.getenv('SATURN_JUPYTER_BASE_DOMAIN')}/proxy/8000/")
%tensorboard --logdir logs/fit --port 8000 --bind_all
# This will generate a hyperlink. Click on that to open TensorBoard!
# (You'll see a 404 error below the link, just ignore that.)

# Call TensorBoard [Not in SaturnCloud]
# Uncomment the next time if you are not in SC
# %tensorboard --logdir logs/fit

https://None/proxy/8000/


Reusing TensorBoard on port 8000 (pid 288), started 0:07:31 ago. (Use '!kill 288' to kill it.)

#### Results and Predictions

Calculate the predictions, save them as a csv, and print them.

In [11]:
# Your code here (use more cells if you need to)
preds = classifier.predict(test)
results = ids.assign(Survived=preds)
results['Survived'] = results['Survived'].apply(lambda row: 1 if row > 0.5 else 0)
results.to_csv('titanic_submission.csv',index=False)
results.head(20)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
5,897,0
6,898,0
7,899,0
8,900,1
9,901,0


Continue to tweak your model until you are happy with the results based on model evaluation.

## Conclusion

Now that you have the `TensorBoard` to help you look at your model, you can better understand how to tweak your model.

How do your predictions compare to what you did last time?

Remember that your "fancier" model may be less accurate... but that is okay if that is the case since we're trying to guard against variance with regularization techniques.